In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/brighton-a-memorable-city/sample_valid_submission.csv
/kaggle/input/brighton-a-memorable-city/training.csv
/kaggle/input/brighton-a-memorable-city/testing.csv
/kaggle/input/brighton-a-memorable-city/additional_training.csv
/kaggle/input/brighton-a-memorable-city/annotation_confidence.csv
/kaggle/input/brighton-a-memorable-city/test_proportions.csv


In [9]:
## Training data

# Get the data from training.csv
dataset = pd.read_csv('/kaggle/input/brighton-a-memorable-city/training.csv')
# Extract the predictions of the images
_y = dataset['prediction']
# Remove the ID and predictions 
_X = dataset.iloc[:247, 1:4609]

In [3]:
from sklearn.impute import KNNImputer

## Extra training data

#Get the data from additional_training.csv
dataset2 = pd.read_csv('/kaggle/input/brighton-a-memorable-city/additional_training.csv')
# Extract the predictions of the images
_y2 = dataset2['prediction']
# Remove the ID and predictions 
_X2 = dataset2.iloc[:2219, 1:4609]

## Defining a KNNImputer to fill in the empty slots in the data
imputer = KNNImputer(n_neighbors=3, missing_values=np.nan)
# Uses the imputer to fill nan data
_X2 = imputer.fit_transform(_X2)
# Convert back to DataFrame
_X2 = pd.DataFrame(_X2)

In [10]:
## Add training data and additional training data together into one
# Numpy concatenation of all training images 
train = np.concatenate([_X,_X2])
# Pandas concatenation of all training predictions
preds = pd.concat([_y,_y2])

In [5]:
## Testing data

#Get the data from testing.csv
dataset3 = pd.read_csv('/kaggle/input/brighton-a-memorable-city/testing.csv')
# Get the ID number of all testing images for output
ID = dataset3['ID']
# Remove the ID column 
test = dataset3.iloc[:11874, 1:4609]

In [11]:
from sklearn.neural_network import MLPClassifier

## Create a new MLPClassifier to predict images
clf = MLPClassifier(solver='lbfgs', alpha = 1e-5, learning_rate = 'adaptive', random_state = 1)
##alpha = 3e-2
## Train classifier on training and additional training images and predictions
clf.fit(train,preds)

MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(100,), learning_rate='adaptive',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='lbfgs',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [12]:
## Predicts whether testing images are memorable or not
results = clf.predict(test)

# Count number of 0s and 1s for predicted data
zero = 0
one = 0
for i in range(int((len(results)/4)+0.5)):
    if(results[i] == 0):
        zero = zero + 1
    else:
        one = one + 1

print("0 = " + str(zero))
print("1 = " + str(one))
        

0 = 1208
1 = 1761


In [8]:
## Outputs predictions into file
sub = pd.DataFrame(zip(ID,results), columns = ['ID','Prediction'])
sub.to_csv("./results2.csv", index = False, header = True)